In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import re
import nltk; nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stefl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
file = open('Claude.txt','r')
claude = file.read()
file.close()
file = open("Ida.txt", 'r')
ida = file.read()
file.close()
file = open ('Youyou_Claire.txt', 'r')
yc = file.read()
file.close()

In [3]:
def preprocess (data):
    if data is not None:
        #exclusions = ['RE:', 'Re:', 're:']
        #exclusions = '|'.join(exclusions)
        data =  data.lower()
        data = re.sub('re:', '', data)
        data = re.sub('-', '', data)
        data = re.sub('_', '', data)
        # Remove data between square brackets
        data =re.sub('\[[^]]*\]', '', data)
        # removes punctuation
        data = re.sub(r'[^\w\s]','',data)
        data = re.sub(r'\n',' ',data)
        data = re.sub(r'[0-9]+','',data)
        # strip html 
        p = re.compile(r'<.*?>')
        data = re.sub(r"\'ve", " have ", data)
        data = re.sub(r"can't", "cannot ", data)
        data = re.sub(r"n't", " not ", data)
        data = re.sub(r"I'm", "I am", data)
        data = re.sub(r" m ", " am ", data)
        data = re.sub(r"\'re", " are ", data)
        data = re.sub(r"\'d", " would ", data)
        data = re.sub(r"\'ll", " will ", data)
        return data
    return 'No Subject'


In [4]:
doc1 = preprocess(claude)
doc2 = preprocess(ida)
doc3 = preprocess(yc)

In [5]:
docs = [doc1, doc2, doc3]

preprocess runs on strings so run first then put into list. if in dataframe use .apply

Next creat term-doc matrix with cv.fit_transform. it also removes stopwords) 

In [6]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')
stop_words = stop_words + ['um', 'im', 'yeah'] 

In [7]:
cv = CountVectorizer(max_df = 0.85, stop_words = stop_words, ngram_range=(1,3), max_features = 50)
# min_df takes out words occuring in less than x documents or x proportion of documents, max_df for word that are too common - occuring in more than x number or proportion of documents
# ngram_range takes min and max ie. (1,2) means unigrams and bigrams. use (1,3) for uni to trigrams
# max_features limits to number of vocab returned 

word_count_vector = cv.fit_transform(docs)

In [8]:
# cv.stop_words_ checks stop words
# cv.vocabulary_ to see what we have

In [9]:
df = pd.DataFrame(word_count_vector.T.todense(),index = cv.get_feature_names(), columns=['doc1', 'doc2', 'doc3'])
df

,doc1,doc2,doc3
american,1,0,15
anymore,2,5,0
back home,0,1,10
card,0,0,12
china,0,0,52
chinese,1,0,21
citizenship,0,0,11
coffee,0,5,3
countries,1,0,8
couple,0,8,2


In [10]:
word_count_vector.shape

(3, 50)

Next, compute IDF values

In [11]:
#train the transfomer 

transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

Note: An extremely important point to note here is that the IDF should always be based on a large corpora and should be representative of texts you would be using to extract keywords. This is why we are using texts from 20,000 stack overflow posts to compute the IDF instead of just a handful. I’ve seen several articles on the Web that compute the IDF using a handful of documents. You will defeat the whole purpose of IDF weighting if its not based on a large corpora as (a) your vocabulary becomes too small and (b) you have limited ability to observe the behavior of words that you do know about.


!!!
Would be ideal to have historic dataset of interviews


However experimenting here to see how models work as realistically want this to be able to work on project by project basis

In [12]:
feature_names = cv.get_feature_names()
tfidf_vectors = transformer.transform(word_count_vector)


In [17]:
first_doc_vectors= tfidf_vectors[0]
first_doc_df = pd.DataFrame(first_doc_vectors.T.todense(), index = cv.get_feature_names(), columns=['Intw 1'])
first_doc_df.sort_values(by=['Intw 1'], ascending=False).head(10)

,Intw 1
difficult,0.401004
made,0.300753
kong,0.300753
know mean,0.300753
train,0.300753
hong kong,0.300753
hong,0.300753
stories,0.250627
hes,0.250627
everyone,0.200502


In [19]:
sec_doc_vectors= tfidf_vectors[1]
sec_doc_df = pd.DataFrame(sec_doc_vectors.T.todense(), index = cv.get_feature_names(), columns=['Intw 2'])
sec_doc_df.sort_values(by=['Intw 2'], ascending=False).head(10)

,Intw 2
sometimes,0.392306
fucking,0.343890
talk,0.326921
life,0.326921
everyone,0.261537
stuff like,0.261537
friends,0.261537
couple,0.261537
like theres,0.196153
soon,0.196153


In [20]:
third_doc_vectors= tfidf_vectors[2]
third_doc_df = pd.DataFrame(third_doc_vectors.T.todense(), index = cv.get_feature_names(), columns=['Intw 3'])
third_doc_df.sort_values(by=['Intw 3'], ascending=False).head(10)

,Intw 3
china,0.695059
stay,0.274471
school,0.240597
chinese,0.213477
journalism,0.160398
card,0.160398
green card,0.160398
friends,0.152484
american,0.152484
citizenship,0.147032


# Caution on small datasets. 

However I think this is a good overview of what word count vectors and tfidf can do to give a flavour of what overall themes and each document says. 

with ngrams the tfidf scores have clearly increased. 

Additionally feel more confident in creating word vectors and controlling the input/ output by using ngrams, min_df, max_df, appending to stopwords etc. 